In [30]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier

In [31]:
kdd = pd.read_csv("data/kddcup_data_10_percent")

In [32]:
protocol_types = pd.unique(kdd.protocol_type.ravel())
protocol_dict = dict(zip(protocol_types,range(protocol_types.size)))

service_types = pd.unique(kdd.service.ravel())
service_dict = dict(zip(service_types,range(service_types.size)))

flag_types = pd.unique(kdd.flag.ravel())
flag_dict = dict(zip(flag_types,range(flag_types.size)))
classification_dict = {"back.": "dos","land.":"dos","neptune.": "dos", "pod.": "dos", "smurf.":"dos","teardrop.":"dos","loadmodule.":"u2r","buffer_overflow.":"u2r","perl.":"u2r","rootkit.":"u2r","ftp_write.":"r2l","guess_passwd.":"r2l","imap.":"r2l","multihop.":"r2l","phf.":"r2l","spy.":"r2l","warezclient.":"r2l","warezmaster.":"r2l","ipsweep.":"probe","nmap.":"probe","portsweep.":"probe","satan.":"probe"}
classification_dict_test = {"back.": "dos","land.":"dos","neptune.": "dos", "pod.": "dos", "smurf.":"dos","teardrop.":"dos","loadmodule.":"u2r","buffer_overflow.":"u2r","perl.":"u2r","rootkit.":"u2r","ftp_write.":"r2l","guess_passwd.":"r2l","imap.":"r2l","multihop.":"r2l","phf.":"r2l","spy.":"r2l","warezclient.":"r2l","warezmaster.":"r2l","ipsweep.":"probe","nmap.":"probe","portsweep.":"probe","satan.":"probe","snmpgetattack.":"dos","apache2.":"dos","udpstorm.":"dos","processtable.":"dos","mailbomb.":"dos","xterm.":"u2r","ps.":"u2r","sqlattack.":"u2r","named.":"r2l","xlock.":"r2l","xsnoop.":"r2l","sendmail.":"r2l","httptunnel.":"r2l","worm.":"r2l","saint.":"probe","mscan.":"probe","snmpguess.":"probe"}



In [33]:
kdd["protocol_type"].replace(protocol_dict, inplace=True)
kdd["service"].replace(service_dict, inplace=True)
kdd["flag"].replace(flag_dict, inplace=True)
kdd["classification"].replace(classification_dict, inplace=True)

In [34]:
kdd_test = pd.read_csv("data/kddcup_test_data")

In [35]:

kdd_test["protocol_type"].replace(protocol_dict, inplace=True)
kdd_test["service"].replace(service_dict, inplace=True)
kdd_test["flag"].replace(flag_dict, inplace=True)
kdd_test["classification"].replace(classification_dict_test, inplace=True)
kdd_test["service"].replace({"icmp":65}, inplace=True)

In [55]:
values = kdd.values
kdd_X_train = values[:,:41]
kdd_y_train = values[:,41]

values_test = kdd_test.values
kdd_X_test = values_test[:,:41]
kdd_y_test = values_test[:,41]





In [42]:
knn = KNeighborsClassifier()

In [43]:
knn.fit(kdd_X_train, kdd_y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [56]:
kdd_predict = knn.predict(kdd_X_test)

In [59]:
#normal,probe,dos,u2r,r2l
normal = [0,0,0,0,0]
probe = [0,0,0,0,0]
dos = [0,0,0,0,0]
u2r = [0,0,0,0,0]
r2l = [0,0,0,0,0]
results_table = [normal,probe,dos,u2r,r2l]
results_table

[[0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0]]

In [66]:
result_hash = {"normal.":0,"probe":1,"dos":2,"u2r":3,"r2l":4}
for x in range(kdd_y_test.size):
    results_table[result_hash[kdd_y_test[x]]][result_hash[kdd_predict[x]]]+=1

In [75]:
results_table = np.array(results_table)
results_table

array([[ 60260,    133,    187,      1,     12],
       [  3157,   2867,    544,      0,      4],
       [ 14290,    111, 223192,      0,      1],
       [    57,      0,      0,     13,      0],
       [  5559,    534,     98,      0,      9]])

In [70]:
sum_predicted = results_table.sum(axis=0)
sum_predicted

array([ 83323,   3645, 224021,     14,     26])

In [71]:
sum_actual = results_table.sum(axis= 1)
sum_actual

array([ 60593,   6572, 237594,     70,   6200])

In [73]:
actual_percent = [float(results_table[0][0])/float(sum_actual[0]),float(results_table[1][1])/float(sum_actual[1]),float(results_table[2][2])/float(sum_actual[2]),float(results_table[3][3])/float(sum_actual[3]),float(results_table[4][4])/float(sum_actual[4])]
actual_percent

[0.9945043156800291,
 0.43624467437614123,
 0.9393839911782285,
 0.18571428571428572,
 0.0014516129032258066]

In [74]:
predicted_percent = [float(results_table[0][0])/float(sum_predicted[0]),float(results_table[1][1])/float(sum_predicted[1]),float(results_table[2][2])/float(sum_predicted[2]),float(results_table[3][3])/float(sum_predicted[3]),float(results_table[4][4])/float(sum_predicted[4])]
predicted_percent


[0.723209678000072,
 0.786556927297668,
 0.9962994540690382,
 0.9285714285714286,
 0.34615384615384615]